In [76]:
import pymysql
import logging
import pandas as pd
logging.basicConfig(level=logging.INFO)

mysql_host = "142.132.134.21"
mysql_prot = 9030
mysql_user = "dw"
mysql_pd = "a64imrJwqKZc9jiU!*aCYi.xnh9*TVhhBmZi"
mysql_db = "dw"
connection = pymysql.connect(host=mysql_host, user=mysql_user, password=mysql_pd, db=mysql_db, port=mysql_prot, charset='utf8')
cursor = connection.cursor()
logging.info("connection created")


def getSqlResult(sql):
    cursor.execute(sql)
    res_rows = cursor.fetchall()
    df = pd.DataFrame(res_rows)
    return df

INFO:root:connection created


# 0x958236266991bc3fe3b77feaacea120f172c0708ad01c7a715b255f218f9313c

In [17]:
txn_hash = '0x958236266991bc3fe3b77feaacea120f172c0708ad01c7a715b255f218f9313c'
txn_detail_sql = """
select a.block_number,a.log_index,a.transaction_hash
    ,a.token_address,b.symbol,b.token_description,b.decimals
    ,a.from_address,c.tag_code
    ,a.to_address,c.tag_code
    ,a.value
    ,a.value/POWER(10, b.decimals) as txn_value
from prod.transfer_event_eth as a
left join prod.token as b
on a.token_address = b.address
left join
(
    select address
        ,max(tag_code) as tag_code
    from prod.tag_address
    group by 1
) as c
on a.from_address = c.address
left join 
(
    select address
        ,max(tag_code) as tag_code
    from prod.tag_address
    group by 1
) as d
on a.to_address = d.address
where a.transaction_hash = '{txn_hash}'
order by log_index asc
""".format(txn_hash=txn_hash)
df = pd.read_sql(txn_detail_sql, con=connection)

/opt/conda/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


# Coinbase Transfer

In [5]:
def get_coinbase_trf(txn_list,start_block,end_block):
    sql ="""
    select a.block_number
        ,a.transaction_hash
        ,max(case when a.miner is not null then a.amount else 0 end) as coinbase_trf_amt
        ,max(b.effective_gas_price * b.gas_used / POWER(10.0, 18)) as gas_fee
    from (
        select a.block_number,a.transaction_hash
            ,a.from_address,a.to_address
            ,b.miner,a.token_address,a.symbol,a.amount
        from dw.dwb_token_transfer_detail_eth_hi as a
        left join prod.blocks as b
        on a.block_number = b.block_number
        and a.to_address = b.miner
        and b.block_number >= {start_block} and b.block_number <= {end_block}
        and b.`chain` = 'eth'
        where a.block_number >= {start_block} and a.block_number <= {end_block}
        and a.transaction_hash in ({txn_list})
    ) as a 
    left join prod.transactions_eth as b
    on a.transaction_hash = b.transaction_hash
    and b.block_number >= {start_block} and b.block_number<= {end_block}
    group by 1,2
    """.format(
        start_block = start_block,
        end_block = end_block,
        txn_list = ",".join('"'+txn+'"' for txn in txn_list)
    )
    df = getSqlResult(sql)
    return df


txn_list = [
    '0xf41775f2a03045511deee450845d6e197a2909ac46c2f3759f2dff699eee5aa1',
    '0xec7b3c821492dd76ac274b7ac37e0b55cabe535ecbd917b740f8d3389c2c3b27',
    '0xc154db3aefe3e7a1d58ee45c2b543faa49e341075588f7e1e113ffb93d74d930'
]
start_block = 15100000
end_block = 15178000
df = get_coinbase_trf(txn_list,start_block,end_block)
df

,0,1,2,3
0,15177999,0xec7b3c821492dd76ac274b7ac37e0b55cabe535ecbd9...,0.517402,0.005718
1,15177999,0xf41775f2a03045511deee450845d6e197a2909ac46c2...,0.231752,0.013493
2,15103060,0xc154db3aefe3e7a1d58ee45c2b543faa49e341075588...,0.000000,0.016140


# Protocal Liquidity

In [80]:
def get_protocol_liquidity(pair_list):
    sql ="""
    select a.account_address as protocol_address
        ,a.token_address
        ,a.`value` * b.price as usd_value
    from dw.dws_token_balance_eth as a
    left join
    (
        select token_address,max(price) as price
        from dw.dwb_token_price_eth_byday_di
        where dt >= DATE_SUB(now(),1)
        group by 1
    ) as b
    on a.token_address = b.token_address
    where a.account_address in ({pair_list_str})
    """.format(
        pair_list_str = ",".join('"'+p[0]+'"' for p in pair_list)
    )
    df = getSqlResult(sql)
    df.set_axis(["protocol_address","token_address","liquidity"],axis=1,inplace=True)

    df_raw=[]
    for i in range(len(pair_list)):
        for j in range(len(pair_list[i])-1):
            df_raw.append([pair_list[i][0],pair_list[i][j+1]])
    df1 = pd.DataFrame(df_raw)
    df1.set_axis(["protocol_address","token_address"],axis=1,inplace=True)

    df_res = pd.merge(
        df,df1,how='inner',
        left_on=['protocol_address','token_address'],
        right_on=['protocol_address','token_address']
    )
    df_res = df_res.groupby(['protocol_address']).sum().reset_index()
    return df_res


pair_list = [
    ('0x760166fa4f227da29ecac3bec348f5fa853a1f3c','0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2','0x0000000000085d4780b73119b644ae5ecd22b376'), 
]
df = get_protocol_liquidity(pair_list)
df

,protocol_address,liquidity
0,0x760166fa4f227da29ecac3bec348f5fa853a1f3c,151234.03069


# Test

In [68]:
pair_list = [
    ('0x5777d92f208679db4b9778590fa3cab3ac9e2168','0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', '0x6b175474e89094c44da98b954eedeac495271d0f'),
    ('0x2db094c50181f39863858f1878545feb26b975d0', '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', '0x0bc529c00C6401aEF6D220BE8C6Ea1667F6Ad93e'), 
    ('0x3416cf6c708da44db2624d63ea0aaef7113527c6', '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', '0xdac17f958d2ee523a2206206994597c13d831ec7'), 
    ('0xd8dec118e1215f02e10db846dcbbfe27d477ac19', '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', '0x6b175474e89094c44da98b954eedeac495271d0f'), 
    ('0xe6ff8b9a37b0fab776134636d9981aa778c4e718', '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', '0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599'), 
    ('0xb20ed18ee19747c0811d57d38b7ab3d71e44c191', '0x6b175474e89094c44da98b954eedeac495271d0f', '0x7Fc66500c84A76Ad7e9c93437bFc5Ac33E2DDaE9'), 
    ('0x48da0965ab2d2cbf1c17c09cfb5cbe67ad5b1406', '0x6b175474e89094c44da98b954eedeac495271d0f', '0xdac17f958d2ee523a2206206994597c13d831ec7')
]
df_raw=[]
for i in range(len(pair_list)):
    for j in range(2):
        df_raw.append([pair_list[i][0],pair_list[i][j+1]])
df1 = pd.DataFrame(df_raw)
df1.set_axis(["protocol_address","token_address"],axis=1,inplace=True)

df_res = pd.merge(df,df1,how='inner',left_on=['protocol_address','token_address'],right_on=['protocol_address','token_address'])
df_res.groupby(['protocol_address']).sum().reset_index()

,protocol_address,market_value
0,0x2db094c50181f39863858f1878545feb26b975d0,9.003840e-06
1,0x3416cf6c708da44db2624d63ea0aaef7113527c6,1.614799e+08
2,0x48da0965ab2d2cbf1c17c09cfb5cbe67ad5b1406,7.680899e+05
3,0x5777d92f208679db4b9778590fa3cab3ac9e2168,8.981315e+08
4,0xb20ed18ee19747c0811d57d38b7ab3d71e44c191,0.000000e+00
5,0xd8dec118e1215f02e10db846dcbbfe27d477ac19,7.531073e+00
6,0xe6ff8b9a37b0fab776134636d9981aa778c4e718,3.507081e+02


In [89]:
int('027f7d0bdb920000',16)/pow(10,18)

0.18